# 🛡️ Manejo de Errores en Agent Framework

## El Problema: ¿Por Qué Validar Errores?

**Escenario sin validación:**
```
Usuario: "¿Quién es el oficial -1?"
  ↓
Función: busca BASE_DATOS[-1] sin preguntar
  ↓
Excepción cruda: KeyError
  ↓
Agente confundido → Mala experiencia
```

## La Solución: 3 Niveles de Defensa

| Nivel | Responsable | Acción |
|-------|-------------|--------|
| **1** | Tu Función | Valida entrada + try/except |
| **2** | @ai_function | Reintenta si falla temp |
| **3** | Agente | Comunica resultado claro |

**Punto clave:** @ai_function NO reemplaza tu validación, la complementa.

---
## Paso 1: Setup del Entorno

In [1]:
import os
import asyncio
from typing import Annotated
from pydantic import Field
from agent_framework import ChatAgent, ai_function
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("✅ Entorno configurado")

✅ Entorno configurado


---
## Paso 2: Definir Base de Datos

In [2]:
# Base de datos de la Flota Estelar
BASE_DATOS_FLOTA = {
    1: {"nombre": "Jean-Luc Picard", "rango": "Capitán", "asignacion": "Enterprise-D"},
    2: {"nombre": "T'Pral", "rango": "Oficial de Ciencias", "asignacion": "Enterprise-D"},
    3: {"nombre": "Worf", "rango": "Jefe de Seguridad", "asignacion": "Enterprise-D"},
    4: {"nombre": "Geordi La Forge", "rango": "Jefe de Ingeniería", "asignacion": "Enterprise-D"},
    5: {"nombre": "Data", "rango": "Oficial de Operaciones", "asignacion": "Enterprise-D"},
}

print(f"✅ Base de datos cargada con {len(BASE_DATOS_FLOTA)} oficiales")

✅ Base de datos cargada con 5 oficiales


---
## Paso 3: Implementar Herramienta con Validación

In [3]:
@ai_function(
    name="consultar_registro_flota",
    description="Busca oficiales de la Flota por ID. Valida entrada y devuelve información clara.",
    max_invocations=5,              # Límite: máximo 5 consultas (anti-bucle)
    max_invocation_exceptions=2     # Framework reintenta si falla <2 veces
)
def consultar_registro_flota(
    id_oficial: Annotated[int, Field(description="ID único del oficial (1-5)")]
) -> str:
    """
    Consulta con validación en 3 niveles:
    1. Tipo de dato
    2. Rango válido
    3. Existencia en BD
    
    @ai_function maneja reintentos si excepciones temporales (BD no disponible, etc)
    """
    
    try:
        # ═══════════════════════════════════════════════════════════
        # NIVEL 1: Validar Tipo de Dato
        # ═══════════════════════════════════════════════════════════
        if not isinstance(id_oficial, int):
            return f"[ERROR] Tipo inválido: esperaba int, recibí {type(id_oficial).__name__}"

        # ═══════════════════════════════════════════════════════════
        # NIVEL 2: Validar Rango
        # ═══════════════════════════════════════════════════════════
        if id_oficial <= 0:
            return f"[ERROR] ID {id_oficial} inválido: debe ser positivo"

        if id_oficial > 1000000:
            return f"[ERROR] ID {id_oficial} demasiado grande (máximo 1000000)"

        # ═══════════════════════════════════════════════════════════
        # NIVEL 3: Validar Existencia en BD
        # ═══════════════════════════════════════════════════════════
        if id_oficial not in BASE_DATOS_FLOTA:
            return f"[NO ENCONTRADO] No existe oficial con ID {id_oficial}"

        # ═══════════════════════════════════════════════════════════
        # EJECUCIÓN: Acceso a Base de Datos
        # ═══════════════════════════════════════════════════════════
        oficial = BASE_DATOS_FLOTA[id_oficial]
        respuesta = (
            f"✅ OFICIAL ENCONTRADO - "
            f"Nombre: {oficial['nombre']}, "
            f"Rango: {oficial['rango']}, "
            f"Asignación: {oficial['asignacion']}"
        )
        return respuesta

    except Exception as e:
        # @ai_function reintentará automáticamente si la excepción es "temporal"
        return f"[ERROR] Excepción en base de datos: {str(e)}"


print("✅ Herramienta definida: consultar_registro_flota")
print("   - Validación en 3 niveles")
print("   - Decorada con @ai_function")
print("   - Manejo de excepciones")

✅ Herramienta definida: consultar_registro_flota
   - Validación en 3 niveles
   - Decorada con @ai_function
   - Manejo de excepciones


### ¿Qué está pasando aquí?

```python
@ai_function(
    max_invocations=5,              # ← Framework previene bucles infinitos
    max_invocation_exceptions=2     # ← Framework reintenta si error temp
)
```

**PERO la función en sí:**
- Valida que `id > 0` ← **TÚ lo haces**
- Valida que `id in BD` ← **TÚ lo haces**
- Captura excepciones con try/except ← **TÚ lo haces**


### 📚 Parámetros de @ai_function - Especificación Completa

Según Microsoft Learn, `@ai_function` proporciona 7 parámetros configurable:

| Parámetro | Tipo | Default | Propósito |
|-----------|------|---------|-----------|
| **name** | `str` | Nombre función | ID único que ve el agente |
| **description** | `str` | Docstring | Explicación para decisiones del agente |
| **approval_mode** | `'always_require' \| 'never_require'` | `None` | Requiere confirmación usuario |
| **max_invocations** | `int` | `None` | Límite de llamadas por conversación |
| **max_invocation_exceptions** | `int` | `None` | Límite de excepciones permitidas |
| **additional_properties** | `dict` | `None` | Metadatos personalizados |
| **func** | `Callable` | Auto | Función a envolver (casos avanzados) |

### 🔄 Comportamiento de Reintentos

**Escenario: `max_invocation_exceptions=2`**

```
Intento 1: ✅ Éxito → Devuelve resultado
Intento 2: ❌ ConnectionError (contador: 1/2) → Framework reintenta automático
Intento 3: ❌ Timeout (contador: 2/2) → Framework reintenta automático
Intento 4: ❌ ConnectionError (contador: 3/2) → ⛔ LÍMITE ALCANZADO
           → El agente recibe mensaje de error del framework
           → El agente puede decidir qué hacer (comunicar error, intentar otra cosa)
```

**Clave:** El framework maneja reintentos sin código adicional

**Conclusión:** Tu código + @ai_function = Defensa completa

Si solo usaras @ai_function sin validación:
- Entrada inválida (`-1`) → No validada → Excepción → @ai_function intenta reintentar infinito ❌

Con tu validación:
- Entrada inválida (`-1`) → `return "[ERROR]..."` → No es excepción → @ai_function no reintenta ✅

---
## Paso 4: Crear Agente

In [4]:
cliente = OpenAIChatClient(
    base_url=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    model_id=AZURE_OPENAI_DEPLOYMENT
)

agente = ChatAgent(
    chat_client=cliente,
    name="AgenteConsultasFlota",
    instructions=(
        "Eres un asistente de consultas de la Flota Estelar. "
        "Tu rol es buscar información de oficiales usando la herramienta disponible. "
        "Si la herramienta devuelve [ERROR], explica el problema al usuario. "
        "Si devuelve [NO ENCONTRADO], comunica que ese oficial no existe. "
        "Sé conciso y profesional."
    ),
    tools=[consultar_registro_flota]
)

print("✅ Agente creado y listo")

✅ Agente creado y listo


---
## Paso 5: Ejecutar Pruebas

In [5]:
async def ejecutar_pruebas():
    """
    Pruebas de los 3 escenarios principales:
    1. Éxito: ID válido que existe
    2. No encontrado: ID válido pero no en BD
    3. Inválido: ID que no cumple validación
    """
    
    pruebas = [
        {
            "tipo": "✅ ÉXITO",
            "consulta": "¿Quién es el oficial 1?",
            "descripcion": "ID válido que existe → devuelve información"
        },
        {
            "tipo": "❌ NO ENCONTRADO",
            "consulta": "¿Existe algún oficial con ID 999?",
            "descripcion": "ID válido pero no en BD → [NO ENCONTRADO]"
        },
        {
            "tipo": "❌ INVÁLIDO",
            "consulta": "Busca el oficial -1",
            "descripcion": "ID negativo → [ERROR] (validación NIVEL 2)"
        },
        {
            "tipo": "✅ ÉXITO",
            "consulta": "Dame info del oficial 3",
            "descripcion": "Otro oficial válido"
        },
    ]
    
    print("="*80)
    print("🧪 SUITE DE PRUEBAS: Validación y Manejo de Errores")
    print("="*80)
    
    for i, prueba in enumerate(pruebas, 1):
        print(f"\n[Prueba {i}] {prueba['tipo']}")
        print(f"  Consulta: {prueba['consulta']}")
        print(f"  Descripción: {prueba['descripcion']}")
        print(f"  Ejecutando...")
        
        resultado = await agente.run(prueba['consulta'])
        print(f"  Respuesta: {resultado.text}")
        print("-" * 80)

await ejecutar_pruebas()

🧪 SUITE DE PRUEBAS: Validación y Manejo de Errores

[Prueba 1] ✅ ÉXITO
  Consulta: ¿Quién es el oficial 1?
  Descripción: ID válido que existe → devuelve información
  Ejecutando...
  Respuesta: El oficial 1 es Jean-Luc Picard, Capitán asignado a la nave Enterprise-D.
--------------------------------------------------------------------------------

[Prueba 2] ❌ NO ENCONTRADO
  Consulta: ¿Existe algún oficial con ID 999?
  Descripción: ID válido pero no en BD → [NO ENCONTRADO]
  Ejecutando...
  Respuesta: No existe ningún oficial con el ID 999 en los registros de la Flota Estelar.
--------------------------------------------------------------------------------

[Prueba 3] ❌ INVÁLIDO
  Consulta: Busca el oficial -1
  Descripción: ID negativo → [ERROR] (validación NIVEL 2)
  Ejecutando...
  Respuesta: El identificador de oficial proporcionado (-1) no es válido. Por favor, asegúrate de ingresar un ID entre 1 y 5.
-----------------------------------------------------------------------------

---
## Resumen: Por Qué Ambos Son Necesarios

### ❌ Si SOLO usas @ai_function (SIN tu validación)
```python
@ai_function(max_invocation_exceptions=2)
def consultar(id):
    return BASE_DATOS[id]  # ← Sin validar id
```

**Problema:** Entrada `-1`
1. @ai_function intenta llamar función
2. `BASE_DATOS[-1]` → KeyError (excepción)
3. @ai_function cuenta exception (1/2)
4. @ai_function reintenta automáticamente
5. `BASE_DATOS[-1]` → KeyError de nuevo
6. Contador (2/2)
7. @ai_function intenta tercera vez
8. Excepción (3/2) → Finalmente falla

❌ Desperdiciaste 2 reintentos en un error de VALIDACIÓN (no temporal)

---

### ✅ Si usas TU VALIDACIÓN + @ai_function (Como en este notebook)
```python
@ai_function(max_invocation_exceptions=2)
def consultar(id):
    if id <= 0:                    # ← TÚ validas
        return "[ERROR]..."        # ← Devuelves string (NO excepción)
    try:
        return BASE_DATOS[id]
    except:
        return "[ERROR]..."
```

**Entrada `-1`:**
1. TÚ validas `if id <= 0`
2. TÚ devuelves `"[ERROR] ID -1 inválido"`
3. @ai_function recibe string (NO excepción)
4. @ai_function NO reintenta (correcto, no es error temporal)
5. Agente recibe: `"[ERROR]..."`
6. Agente comunica al usuario claramente

✅ Validación clara + @ai_function se enfoca en reintentos REALES

---

## Checklist: Implementa Esto en TUS Herramientas

- [ ] ¿Validas tipos de entrada (isinstance)?
- [ ] ¿Verificas rangos/valores válidos (> 0, < 100, etc)?
- [ ] ¿Compruebas existencia en BD/recurso antes de acceder?
- [ ] ¿Tienes try/except para excepciones runtime?
- [ ] ¿SIEMPRE devuelves strings (nunca excepciones crudas)?
- [ ] ¿Usas prefijos claros para clasificar errores ([ERROR], [NO ENCONTRADO], ✅)?
- [ ] ¿Decoraste con @ai_function y configuraste parámetros?
- [ ] ¿Testeaste casos: válido, inválido, no encontrado, excepción?